In [3]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    
    from gensim.models.doc2vec import Doc2Vec, TaggedDocument
    from nltk.tokenize import word_tokenize
    import nltk
    nltk.download('punkt')
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

[nltk_data] Downloading package punkt to /home/srinivas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loaded  .. . . . . . . .


In [4]:
df = pd.read_excel("elastic_Search_Data.xlsx")
print("read")

read


In [5]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN


In [6]:
answers=list(df["para_text"])

In [7]:
answers

['ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'Geriatrics (> 65 years of age) : No overall differences in safety or effectiveness were observed between these subjects and younger subjects, and other reported clinical experience has not identified differences in responses between the elderly and younger patients, but greater sensitivity of some older individuals cannot be ruled out. ',
 'Pediatrics (6 to 17 years of age): \n•\tHypertension\nATACAND is indicated for the treatment of essential hypertension in children and adolescents 6 to 17 years of age (se

In [353]:
tagged_data = [TaggedDocument(words=word_tokenize(word.lower()), tags=[str(i)]) for i, word in enumerate(answers)]

In [380]:
ml_vec= Doc2Vec(tagged_data,vector_size=512, min_count=1, epochs=2,dm=0)

In [381]:
type(ml_vec[0])

numpy.ndarray

In [382]:
len(ml_vec[5])

512

In [383]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "ml_vector":{
         "type":"dense_vector",
         "dims":512
      } 
    }
   }
}

In [384]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [400]:
#creating the database
IndexName = 'doc2vezhxhjhjjcsahhgjvj'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [401]:
my

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'doc2vezhxhjhjjcsahhgjvj'}

In [402]:
df.columns

Index(['filename', 'country', 'Product name', 'Refined Product Name',
       'para_text', 'Possible Questions', 'Comment'],
      dtype='object')

In [403]:
df22 = df.to_dict('records')
df22

[{'filename': 'Candesartan_PM_Reviewed.docx',
  'country': 'Canada',
  'Product name': 'Candesartan cilexetil',
  'Refined Product Name': 'Atacand (Candesartan Cilexetil)',
  'para_text': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'Possible Questions': 'What is ATACAND used for\nWhat is ATACAND used for in Canada\nHow can ATACAND be used\nHow can ATACAND be used in Canada\nCan ATACAND be used with calcium channel blockers\nCan ATACAND be used with calcium channel blockers in Canada\nCan ATACAND (candesartan cilexetil) be indicated for heartfailure',
  

In [404]:
len(df22)

99

In [405]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'ml_vector':ml_vec[c]})

In [406]:
arr

[{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'ml_vector': array([ 5.98607061e-04, -7.09159533e-04,  2.65734008e-04, -3.38094775e-04,
          3.35054996e-04, -2.57863750e-04,  3.12504388e-04,  3.32685828e-04,
         -8.31740559e-04,  2.59764609e-04,  9.25826724e-04,  4.79320704e-04,
         -1.76787711e-04, -1.29400767e-04,  4.52000473e-04, -3.76548967e-04,
         -9.26345121e-04, -3.81258324e-05, -1.93977656e-04, -8.84548761e-04,
          6.31013769e-04, -1.73831664e-04, -4.50199004e-04, -8.24840681e-04,
          8.10411002e-04, -6.9

In [407]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': array([ 5.98607061e-04, -7.09159533e-04,  2.65734008e-04, -3.38094775e-04,
         3.35054996e-04, -2.57863750e-04,  3.12504388e-04,  3.32685828e-04,
        -8.31740559e-04,  2.59764609e-04,  9.25826724e-04,  4.79320704e-04,
        -1.76787711e-04, -1.29400767e-04,  4.52000473e-04, -3.76548967e-04,
        -9.26345121e-04, -3.81258324e-05, -1.93977656e-04, -8.84548761e-04,
         6.31013769e-04, -1.73831664e-04, -4.50199004e-04, -8.24840681e-04,
         8.10411002e-04, -6.94008195e

In [408]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [409]:
resp

{'_index': 'doc2vezhxhjhjjcsahhgjvj',
 '_type': '_doc',
 '_id': '98',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 98,
 '_primary_term': 1}

In [410]:
#testing
que = "what is ATACAND?"
que_vec=ml_vec.infer_vector([que])

In [411]:
len(que_vec)

512

In [412]:
que_vec

array([ 3.15035722e-04, -5.79029555e-04, -9.44216561e-04, -1.38694691e-06,
       -6.79627876e-04,  4.54222638e-04,  1.38022020e-04,  7.63220014e-04,
       -7.28773535e-04, -7.60860101e-04,  2.03741249e-04, -6.89110893e-04,
        3.77610442e-04, -6.95577764e-04,  7.37339084e-04,  3.41325096e-04,
       -4.56242851e-04, -5.48528624e-04,  8.03434814e-04,  1.11471491e-04,
        3.87705513e-05, -2.25673532e-04,  5.05965261e-04,  1.81360258e-04,
        3.14649398e-04, -6.86349173e-04,  1.54383219e-04, -2.75991450e-04,
        9.49642912e-04,  3.38954909e-04, -3.97279189e-04, -7.98199559e-04,
        3.33440432e-04, -4.83590964e-04,  7.14634254e-04,  6.48024201e-04,
        5.32883045e-04, -6.90151181e-04,  3.63973348e-04, -7.63363088e-04,
        8.18377011e-04,  3.49509100e-05,  5.71675366e-04,  3.60302249e-04,
       -2.07799108e-04,  4.31048044e-04,  1.74232890e-04,  4.44095349e-04,
        4.97047731e-04, -4.32871093e-05,  8.70557677e-04, -6.85648934e-04,
        3.18761828e-04,  

In [415]:

script_query = {
"script_score": {
"query": {"match_all":{}},
"script": {
"source": "cosineSimilarity(params.query_vector, doc['ml_vector']) ",
"params": {"query_vector": que_vec }
}}}

In [416]:
es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

RequestError: RequestError(400, 'search_phase_execution_exception', 'script_score script returned an invalid score [-0.012572484] for doc [0]. Must be a non-negative score!')